# TwitterAPI

Adapted from [Brian Spiering's code](https://github.com/brianspiering/fun_with_twitter/blob/master/twitter_stream_processing.ipynb) and his [source](http://geduldig.github.io/TwitterAPI/faulttolerance.html), using TwitterAPI (docs found [here](https://github.com/geduldig/TwitterAPI)).

In [1]:
reset -fs

In [2]:
import twitter

In [21]:
twitter.Api?

In [3]:
from itertools import islice
from time import sleep

#from credentials import credentials
from ftfy import fix_text
from textblob import TextBlob
from TwitterAPI import TwitterAPI

/Users/brynstark/anaconda/envs/py3_nlp/lib/python3.5/site-packages/nltk/decorators.py:59: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  regargs, varargs, varkwargs, defaults = inspect.getargspec(func)


In [4]:
from TwitterAPI import TwitterRequestError
from TwitterAPI import TwitterConnectionError

In [5]:
my_consumer_key = '7YlWB6UkgG8UcdHMN2GpLJLZ0'
my_consumer_secret = 'bzVsCkJVKudFDbPUklPkZgpl01V1tH48nT6xb6pTQAJRcbVSzH'
my_access_token = '738091212-F84xHfpB40qVdXdeDxQBmudefWEaAPLmRxofDi92'
my_access_token_secret = 'ukgQosQL5FmncQSROBxTQ2BY84stqdUXE5qdAkDP9LcM8'

In [6]:
api = TwitterAPI(my_consumer_key, my_consumer_secret, my_access_token, my_access_token_secret)
r = api.request('search/tweets', {'q':'pizza'})
print(r.status_code)

200


In [7]:
def tweets(query):
    "A generator of tweets"
    while True:
        try:
            iterator = api.request('search/tweets', {'q':query}).get_iterator()
            for item in iterator:
                if 'text' in item:
                    yield((item['text'], item['created_at']))
                elif 'disconnect' in item:
                    event = item['disconnect']
                    if event['code'] in [2,5,6,7]:
                        # something needs to be fixed before re-connecting
                        raise Exception(event['reason'])
                    else:
                        # temporary interruption, re-try request
                        break
        except TwitterRequestError as e:
            if e.status_code < 500:
                # something needs to be fixed before re-connecting
                raise
            else:
                # temporary interruption, re-try request
                pass
        except TwitterConnectionError:
            # temporary interruption, re-try request
            pass

In [8]:
testing = tweets(query='San Diego')
list(islice(testing, 5))

/Users/brynstark/anaconda/envs/py3_nlp/lib/python3.5/site-packages/IPython/core/formatters.py:90: DeprecationWarning: DisplayFormatter._ipython_display_formatter_default is deprecated: use @default decorator instead.
  def _ipython_display_formatter_default(self):
/Users/brynstark/anaconda/envs/py3_nlp/lib/python3.5/site-packages/IPython/core/formatters.py:96: DeprecationWarning: DisplayFormatter._formatters_default is deprecated: use @default decorator instead.
  def _formatters_default(self):
/Users/brynstark/anaconda/envs/py3_nlp/lib/python3.5/site-packages/IPython/core/formatters.py:675: DeprecationWarning: PlainTextFormatter._deferred_printers_default is deprecated: use @default decorator instead.
  def _deferred_printers_default(self):
/Users/brynstark/anaconda/envs/py3_nlp/lib/python3.5/site-packages/IPython/core/formatters.py:667: DeprecationWarning: PlainTextFormatter._singleton_printers_default is deprecated: use @default decorator instead.
  def _singleton_printers_default(s

[('Ready to rock San Diego? @amygrant at Embarcadero Marina Park South on Aug 20! https://t.co/0YN9060JO5 https://t.co/L6x3QOiv3t',
  'Sun Jul 10 22:56:29 +0000 2016'),
 ('"My daddy San Francisco, Momma San Diego. You mix that \'Cisco with that SoCal make a \'Brae bama" 💁🏼 #ifiwasbeyonce',
  'Sun Jul 10 22:56:28 +0000 2016'),
 ('RT @MLBNetworkRadio: The USA is lined up, minutes before the 2016 @SIRIUSXM Futures Game from PETCO Park in San Diego https://t.co/TyjUPNeT…',
  'Sun Jul 10 22:56:25 +0000 2016'),
 ('RT @ARizzo44: San Diego here we come! @benzobrist18 @Addison_Russell @KrisBryant_23 @JArrieta34 @DexterFowler @JLester34 @Cubs https://t.co…',
  'Sun Jul 10 22:56:23 +0000 2016'),
 ('RT @ARizzo44: San Diego here we come! @benzobrist18 @Addison_Russell @KrisBryant_23 @JArrieta34 @DexterFowler @JLester34 @Cubs https://t.co…',
  'Sun Jul 10 22:56:22 +0000 2016')]

It works! But it's safe to say there's more filtering necessary.

In [49]:
# api.request('statuses/filter', {'track': 'bart', 'locations': ['-122.75,36.8','-121.75,37.8']})

In [53]:
api.request?

In [47]:
def tweets(query, location):
    "A generator of tweets"
    while True:
        try:
            iterator = api.request('search/tweets', {'q': query, 'locations': location}).get_iterator()
            for item in iterator:
                if 'text' in item:
                    yield(item.keys())
                elif 'disconnect' in item:
                    event = item['disconnect']
                    if event['code'] in [2,5,6,7]:
                        # something needs to be fixed before re-connecting
                        raise Exception(event['reason'])
                    else:
                        # temporary interruption, re-try request
                        break
        except TwitterRequestError as e:
            if e.status_code < 500:
                # something needs to be fixed before re-connecting
                raise
            else:
                # temporary interruption, re-try request
                pass
        except TwitterConnectionError:
            # temporary interruption, re-try request
            pass

In [48]:
testing = tweets(query='bart train', location='-122.75,36.8,-121.75,37.8')
list(islice(testing, 1))

[dict_keys(['truncated', 'id', 'coordinates', 'entities', 'contributors', 'user', 'in_reply_to_user_id', 'favorited', 'lang', 'text', 'in_reply_to_screen_name', 'in_reply_to_status_id', 'favorite_count', 'source', 'id_str', 'retweeted_status', 'retweeted', 'geo', 'created_at', 'retweet_count', 'is_quote_status', 'in_reply_to_status_id_str', 'in_reply_to_user_id_str', 'place', 'metadata'])]

In [49]:
def tweets(query=None, location=None):
    "A generator of tweets"
    while True:
        try:
            # Query and/or location can be entered as inputs in tweet()
            if location == None:
                iterator = api.request('search/tweets', {'q': query}).get_iterator()
            elif query == None:
                iterator = api.request('search/tweets', {'locations': location}).get_iterator()
            else:
                iterator = api.request('search/tweets', {'q': query, 'locations': location}).get_iterator()
            for item in iterator:
                if 'text' in item:
                    yield((item['text'], item['created_at'], item['lang']))
                elif 'disconnect' in item:
                    event = item['disconnect']
                    if event['code'] in [2,5,6,7]:
                        # something needs to be fixed before re-connecting
                        raise Exception(event['reason'])
                    else:
                        # temporary interruption, re-try request
                        break
        except TwitterRequestError as e:
            if e.status_code < 500:
                # something needs to be fixed before re-connecting
                raise
            else:
                # temporary interruption, re-try request
                pass
        except TwitterConnectionError:
            # temporary interruption, re-try request
            pass

In [50]:
testing = tweets(query='bart oakland blacklivesmatter')
list(islice(testing, 5))

[('RT @MichelLevinSUN: Exit BART in Oakland. Two helicopters circling overhead like vultures. Night of protest in #Oakland. Peace, love, #blac…',
  'Fri Jul 08 04:32:38 +0000 2016',
  'en'),
 ('RT @MichelLevinSUN: Exit BART in Oakland. Two helicopters circling overhead like vultures. Night of protest in #Oakland. Peace, love, #blac…',
  'Fri Jul 08 03:49:05 +0000 2016',
  'en'),
 ('Exit BART in Oakland. Two helicopters circling overhead like vultures. Night of protest in #Oakland. Peace, love, #blacklivesmatter',
  'Fri Jul 08 03:46:32 +0000 2016',
  'en'),
 ('On BART, seeing all these young POC heading to the Oakland #BlackLivesMatter, waiting for the 12th St stop to get off this train and gather.',
  'Fri Jul 08 02:47:30 +0000 2016',
  'en'),
 ('RT @yanni_sayz: Tonight 12th St/Civic Center Oakland stop on BART come out  #Oakland #BlackLivesMatter #AltonSterling https://t.co/4I4B6soB…',
  'Fri Jul 08 01:44:41 +0000 2016',
  'en')]

In [ ]:
# Just for safe keeping, if needed later:
# sf_location='-122.75,36.8,-121.75,37.8'

Using the code above, we're getting a batch of the data, not collecting it as it streams in real time.

__Let's print the tweets out as they come in...__

In [64]:
try:
    r = api.request('statuses/filter', {'track':'train'})
    for item in r:
        if item['lang'] == 'en':
            print(item['text'], item['created_at']) #item['lang']))
        else:
            continue
except TwitterRequestError as e:
        if e.status_code < 500:
            # something needs to be fixed before re-connecting
            raise
        else:
            # temporary interruption, re-try request
            pass
except TwitterConnectionError:
    # temporary interruption, re-try request
    pass

Gotta get back on the healthy eating train and put on my big girl panties to finish off #c25k. Sun Jul 10 23:41:32 +0000 2016
RT @JayCostTWS: Where Pence has grossly miscalculated is in the premise that anybody can be the voice of reason on the Trump Train. Sun Jul 10 23:41:33 +0000 2016
RT @rumpfshaker: This applies to every Republican who is supporting Trump. You can't fix him. He'll never get better.  https://t.co/sIFIKNJ… Sun Jul 10 23:41:35 +0000 2016
RT @rumpfshaker: This applies to every Republican who is supporting Trump. You can't fix him. He'll never get better.  https://t.co/sIFIKNJ… Sun Jul 10 23:41:37 +0000 2016


KeyboardInterrupt: 

That's fine, but I'll need to write the tweets to a file and then transfer them to Postgres, or load them directly into Postgres. Thinking about how we're going to connect the BART schedule and the Tweets, the pieces of information that it will make most sense to join on is the Tweet's timestamp and the BART train's time closest to that Tweet (the next train, ideally, not one that's already passed).

__See the ipython notebook entitled 'Collecting_streaming_tweets.ipynb'__